## Практическое задание 6. Градиентный бустинг ~~своими руками~~

Поздравляю! Это финальное упражнение в нашем курсе. Проявите все своё старание, терпение и опыт, чтобы выполнить его.  
Теперь вы многое знаете из машинного обучения и для вас не составит сложности попробовать разные алгоритмы, новые библиотеки и применить их к реальной задаче.

__Задание 1. (0.5 балла)__

Мы будем использовать данные соревнования [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk/data).  

* Загрузите таблицу **application_train.csv**;
* Запишите в Y столбец с целевой переменной;
* Удалите ненужные столбцы (для этого воспользуйтесь описанием);
* Определите тип столбцов и заполните пропуски - стратегия произвольная;
* Разбейте выборку в соотношении 70:30 с random_state=0.

Так как в данных значительный дисбаланс классов, в качестве метрики качества везде будем использовать площадь под precision-recall кривой.

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv("application_train.csv")
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
Y = data['TARGET']

In [7]:
data.drop('SK_ID_CURR', axis=1, inplace=True)
data.drop('TARGET', axis=1, inplace=True)
data.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
sum(data.dtypes == 'object') + sum(data.dtypes == 'int64') + sum(data.dtypes == 'float64')

120

In [9]:
def transform_data(data):
    columns = data.select_dtypes(include=['object']).columns
    data[columns] = data[columns].fillna('nan')
    columns = data.select_dtypes(include=['int64', 'float64']).columns
    data[columns] = data[columns].fillna(0)
    return data
data = transform_data(data)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, Y, train_size=0.7, random_state=0)

__Задание 2. (1.5 балла)__

Также мы будем использовать две реализации градиентного бустинга: [LightGBM](https://lightgbm.readthedocs.io/en/stable/Python-API.html) и [Catboost](https://catboost.ai/en/docs/), которые вам необходимо самостоятельно изучить и установить, используя команды:  
`!pip install lightgbm`  
`!pip install catboost`  
Обучите реализации градиентного бустинга LightGBM и Catboost на вещественных признаках без подбора параметров. 
Почему получилась заметная разница в качестве? 

В этом и последующих экспериментах необходимо измерять время обучения моделей.

In [3]:
#!pip install lightgbm

In [4]:
#!pip install catboost

In [72]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import average_precision_score
import time

In [83]:
df = data.copy()
df = df.select_dtypes(include=['int64', 'float64'])
X_train, X_test, y_train, y_test = train_test_split(df, Y, train_size=0.7, random_state=0)

In [80]:
def best_parametrs(model, param_grid, X_train, y_train):

    gridsearch = GridSearchCV(model, param_grid, scoring='average_precision', cv=3)
    t0 = time.time()
    gridsearch.fit(X_train, y_train)
    t1 = time.time()
    return gridsearch.best_params_, t1 - t0

In [81]:
def metric(model, X_train, X_test, y_train, y_test):
    t0 = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    t1 = time.time()
    return average_precision_score(y_test, y_pred), t1 - t0

#### Обучение LightGBM

In [84]:
model = LGBMClassifier(verbose=-100)
metric_LGBM_e, t = metric(model, X_train, X_test, y_train, y_test)

print('LightGBM new:', metric_LGBM_e)
print('Время выполнения:', t)

LightGBM new: 0.23159230793933594
Время выполнения: 7.1419079303741455


#### Обучение CatBoost

In [85]:
model = CatBoostClassifier(logging_level='Silent')
metric_LGBM_e, t = metric(model, X_train, X_test, y_train, y_test)

print('LightGBM new:', metric_LGBM_e)
print('Время выполнения:', t)

LightGBM new: 0.23259848653401144
Время выполнения: 82.01798987388611


**Ответ:** для LightGBM требуется дополнительная настройка гиперпараметров, гиперпараметры для Catboost настраивать не требуется

__Задание 3. (2 балла)__

Подберите с CV=3 оптимальные параметры алгоритмов, изменяя:

* глубину деревьев;
* количество деревьев;
* темп обучения;
* оптимизируемый функционал.

Проанализируйте соотношения глубины и количества деревьев в зависимости от алгоритма.

#### Поиск наилучших параметров для LightGBM

In [86]:
from sklearn.model_selection import GridSearchCV

param_grid = {
        'max_depth': [2, 4, 6],
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.01, 0.1, 0.5],
        'objective': ['binary']
}
model = LGBMClassifier(verbose=-100)
bests, t = best_parametrs(model, param_grid, X_train, y_train)
print('best params: ', bests)
print('Время выполнения:', t)

KeyboardInterrupt: 

In [ ]:
model = LGBMClassifier(verbose=-100)
metric_LGBM_e, t = metric(model, X_train, X_test, y_train, y_test)

print('LightGBM new:', metric_LGBM_e)
print('Время выполнения:', t)

In [57]:
t0 = time.time()
model = LGBMClassifier(learning_rate=0.1, max_depth=6, n_estimators=100, objective='binary', verbose=-100)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, 1]
t1 = time.time()
metric_LGBM = average_precision_score(y_test, y_pred)
print('LightGBM', metric_LGBM)
print('Время выполнения:', t1 - t0)

LightGBM 0.23185290523301771
Время выполнения: 7.206645250320435


#### Поиск наилучших параметров для CatBoost

In [32]:
param_grid = {
    'depth': [2, 4, 6],
    'iterations': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.5],
    'loss_function': ['Logloss']
}

model = CatBoostClassifier(logging_level='Silent')
bests, t = best_parametrs(model, param_grid, X_train, y_train)
print('best params: ', bests)
print('Время выполнения:', t)

best params:  {'depth': 2, 'iterations': 100, 'learning_rate': 0.5, 'loss_function': 'Logloss'}
Время выполнения: 298.55480456352234


In [52]:
t0 = time.time()
model = CatBoostClassifier(depth=2, iterations=100, learning_rate=0.5, loss_function='Logloss', logging_level='Silent')
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, 1]
t1 = time.time()
metric_CatBoost = average_precision_score(y_test, y_pred)
print('Catboost', metric_CatBoost)
print('Время выполнения:', t1 - t0)

Catboost 0.22478943259099962
Время выполнения: 5.405036449432373


In [51]:
#depth = [2, 6, 9]
#for i in range(3):
#    t0 = time.time()
#    model = CatBoostClassifier(depth=depth[i], iterations=100, learning_rate=0.5, loss_function='Logloss', logging_level='Silent')
#    model.fit(X_train, y_train)
#    y_pred = model.predict_proba(X_test)[:, 1]
#    t1 = time.time()
#    
#    print('Catboost', average_precision_score(y_test, y_pred))
#    print('Время выполнения:', t1 - t0)

Catboost 0.22478943259099962
Время выполнения: 5.695862531661987
Catboost 0.22370888020094276
Время выполнения: 9.272752285003662
Catboost 0.19789003339691885
Время выполнения: 18.065347909927368


__Задание 4. (3.5 балла)__

Добавьте категориальные признаки к вещественным следующими способами:

* как OHE признаки;
* как счетчики со сглаживанием.

При подсчете счетчиков запрещается использование циклов. 

На получившихся датасетах подберите параметры у каждого из алгоритмов. Как меняется время, необходимое для обучения модели в зависимости от способа кодирования? Сравните полученные результаты с встроенными методами обработки категориальных признаков. 

In [46]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
cat = data.select_dtypes(include=['object'])

In [49]:
encoder = OneHotEncoder()
encoder_df = pd.DataFrame(encoder.fit_transform(cat).toarray())
df_e = df.join(encoder_df)

In [50]:
df_e

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,136,137,138,139,140,141,142,143,144,145
0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,-2437,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,-1982,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
307507,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,-4090,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
307508,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,-5150,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
307509,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,-931,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [66]:
X_train, X_test, y_train, y_test = train_test_split(df_e, Y, train_size=0.7, random_state=0)

In [67]:
param_grid = {
        'max_depth': [2, 4, 6],
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.01, 0.1, 0.5],
        'objective': ['binary']
}
bests, t = best_parametrs(model, param_grid, X_train, y_train)
print('best params: ', bests)
print('Время выполнения:', t)

best params:  {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'objective': 'binary'}
Время выполнения: 428.09190559387207


In [76]:
model = LGBMClassifier(learning_rate=0.1, max_depth=4, n_estimators=100, objective='binary', verbose=-100)
metric_LGBM_e, t = metric(model, X_train, X_test, y_train, y_test)

print('LightGBM new:', metric_LGBM_e)
print('Время выполнения:', t)

LightGBM new: 0.23428448818935865
Время выполнения: 8.46998143196106


In [77]:
bests, t = best_parametrs(X_train, y_train)
print('best params: ', bests)
print('Время выполнения:', t)

KeyboardInterrupt: 

In [64]:
model = CatBoostClassifier(depth=2, iterations=100, learning_rate=0.5, loss_function='Logloss', logging_level='Silent')

metric_CatBoost_e = average_precision_score(y_test, y_pred)
print('Catboost new', metric_CatBoost_e)
print('Время выполнения:', t1 - t0)

Catboost new 0.230936821541533
Время выполнения: 6.156075716018677


__Задание 5. (1 балл)__

Реализуйте блендинг подобранных в предыдущем задании моделей и сравните качество.

__Задание 6. (1.5 балла)__

В задании 3 вы подобрали гиперпараметры для LightGBM и CatBoost на вещественных признаках. Визуализируйте важности признаков, посчитанные этими алгоритмами, в виде горизонтального bar-plot (отсортируйте признаки по убыванию важности, подпишите названия признаков по оси y).

Для каждого из двух алгоритмов удалите неважные признаки (обычно по bar-plot хорошо видно порог на важность, с которого начинается хвост неважных признаков) и обучите ту же модель на получившихся данных. Сильно ли упало качество при удалении признаков, которые модель считает неважными?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ